In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

### Data preprocessing

In [16]:
df=pd.read_csv("all_clean.csv")
df2=pd.read_csv("myseat_clean.csv")

start_shape=(df.shape)
print("Train dataset shape: ", start_shape)
print("Test dataset shape:  ", df2.shape)
trainset=df.shape[0]
testset=df2.shape[0]

# Join Test, and Train set
df_set=df[:trainset]
df_set=df_set.append(df2)
df_set.shape
df=df_set
print("Join dataset shape:  ",df_set.shape)
df.reset_index()

from sklearn import preprocessing

status_cat={'Hiányos':0, 'Futómű hibás':0, 'Elektronika hibás':0, 'Sérült':1, 'Enyhén sérült':2, 'Sérülésmentes':3,
            'Normál':4, 'Megkímélt':5, 'Kitűnő':6, 'Újszerű':7}
df['status']=df['status'].replace(status_cat)

air_cat={'Manuális klíma':'MK', 'Digitális többzónás klíma':'DTK', 'Digitális kétzónás klíma':'DKK', 'Digitális klíma':'DK',
         'Automata klíma':'AK', 'Nincs':'N'}
df['airc']=df['airc'].replace(air_cat)


status_cat={'Okmányok nélkül':0,
            'Forgalomból ideiglenesen kivont, külföldi okmányokkal':1,
            'Forgalomból ideiglenesen kivont, magyar okmányokkal':2,
            'Lejárt külföldi okmányokkal':3,
            'Lejárt magyar okmányokkal':4,
            'Érvényes külföldi okmányokkal':5,
            'Érvényes magyar okmányokkal':6}


def cat_reducer(df,field, count):
    #Category dimension reduction function
    
    reduce_dict = df.groupby(field)['kod'].nunique().to_dict()
    l = []
    l.append(reduce_dict)
    reduce_df = pd.DataFrame(l)
    reduce_df = reduce_df.T
    reduce_df.sort_values(by = 0, ascending = False)
    
    #Kategóriák listába rendezése
    flist=reduce_df[reduce_df[0] < count].index.tolist()
    #Csere egyébre
    
    out = df[field] = df[field].replace(flist, 'Egyéb')
    dd = df.groupby(field)['kod'].nunique().to_dict()
    l = []
    l.append(dd)
    l
    return out

#Dimension reduction
df['airc'] = cat_reducer(df, 'airc', 100)
df['fuel'] = cat_reducer(df, 'fuel', 100)
df['gearset'] = cat_reducer(df, 'gearset', 100)

df=df.reset_index()

df.index.name = 'Id'

#One hot coding
def onehot(col_list):
    global df
    while len(col_list) !=0:
        col=col_list.pop(0)
        data_encoded = pd.get_dummies(df[col], prefix=col)
        df = pd.merge(df, data_encoded, on='Id')
        df = df.drop(columns=col)
    
    

onehot(['name0','fuel'])
drop_list = ['name', 'look', 'gearset', 'airc', 'run','paper']
df = df.drop(drop_list, axis=1)

#New features

# Price Rounding
df['price'] = round(df['price'],-4)
# Age Calculation
df['age'] = 2019-df['year']

df['load'] = df['fullweight']-df['selfweight']
df['ca'] = round(((df['gearsize'] / df['power']) * df['load']), 0)


#Handle Skewness
df['price_log'] = np.log(df['price'])
df['clock_log'] = np.log(df['clock'])



#Train és Test set split
df2=df[trainset:]
df=df[:trainset]

print("My dataset shape: ",df2.shape)
print(trainset)
print("All_dataset Shape: ",df.shape)
df=df.dropna(subset=['price'], how='all')

df['year'] = df['age']
df['price'] = round(df['price'],-4)

# Drop attributes
df=df.dropna()
dropindex = df[df['year'] > 21.0].index
df=df.drop(dropindex,axis=0)

dropindex = df[df['clock'] > 500000].index
df=df.drop(dropindex,axis=0)

dropindex = df[df['price'] < 250000].index
df=df.drop(dropindex,axis=0)

dropindex = df[df['age'] == 0].index
df=df.drop(dropindex,axis=0)


def custom_round(x, base=5):
    return int(base * round(x/base))

df['power'] = df['power'].apply(lambda x: custom_round(x, base=5))

end_shape = df.shape
print('Drop data:', (1-(end_shape[0]/start_shape[0]))*100,' percent')

df2.to_csv('seat_dataset.csv')
df.to_csv('all_moddataset.csv')

Train dataset shape:  (26406, 19)
Test dataset shape:   (1, 18)
Join dataset shape:   (26407, 19)
My dataset shape:  (1, 29)
26406
All_dataset Shape:  (26406, 29)
Drop data: 7.373324244489887  percent
